### Deliverable 1: Preprocessing the Data for a Neural Network

In [4]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
import kerastuner as kt

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

ModuleNotFoundError: No module named 'kerastuner'

In [ ]:
pip install --upgrade tensorflow

In [ ]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN','NAME'],axis=1)
#drop additional columns for deliverable 3
application_df = application_df.drop(['USE_CASE','STATUS'],axis=1)
application_df.head()

In [ ]:
# Determine the number of unique values in each column.
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
application_df[application_cat].nunique()


In [ ]:
# Look at APPLICATION_TYPE value counts for binning
application_counts = application_df.APPLICATION_TYPE.value_counts()
application_counts

In [ ]:
# Visualize the value counts of APPLICATION_TYPE
application_counts.plot.density()

In [ ]:
# Determine which values to replace if counts are less than ...?
replace_APPLICATION_TYPE = list(application_counts[application_counts < 400].index)

# Replace in dataframe
for app in replace_APPLICATION_TYPE:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

In [ ]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df.CLASSIFICATION.value_counts()
classification_counts.tail(2000)

In [ ]:
# Visualize the value counts of CLASSIFICATION
classification_counts.plot.density()

In [ ]:
# Determine which values to replace if counts are less than ..?
replace_class = list(classification_counts[classification_counts < 500].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

In [ ]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
application_cat


In [ ]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

In [ ]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df,left_index=True, right_index=True)
application_df = application_df.drop(application_cat,1)
application_df.head()

In [ ]:
# Split our preprocessed data into our features and target arrays
y = application_df.IS_SUCCESSFUL
X = application_df.drop(columns=["IS_SUCCESSFUL"])

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### Deliverable 2: Compile, Train and Evaluate the Model

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 300
hidden_nodes_layer2 = 150
#hidden_nodes_layer3 = 6
#hidden_nodes_layer4 = 10
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# third hidden layer
#nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Fourth hidden layer
#nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="sigmoid"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

In [ ]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "Checkpoints/weights.{epoch:02d}.hdf5"

In [ ]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every epoch
#cp_callback = ModelCheckpoint(
 #   filepath=checkpoint_path,
  #  verbose=1,
   # save_weights_only=True,
    #save_freq=5)

In [ ]:
# Train the model
#fit_model = nn.fit(X_train_scaled,y_train,epochs=10,callbacks=[cp_callback])
fit_model = nn.fit(X_train_scaled,y_train,epochs=10)

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Deliverable 3 test

In [ ]:
##Copy dataframe
application_dfd3 = application_df.copy()


##Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid','selu','elu','softmax','softplus','softsign','exponential'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=10,
        step=2), activation=activation, input_dim=38))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=15,
            step=2),
            activation=activation))
    
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_model

In [ ]:
##Create Tuner
tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2
)

In [ ]:
# Split our preprocessed data into our features and target arrays
yd3 = application_dfd3.IS_SUCCESSFUL
Xd3 = application_dfd3.drop(columns=["IS_SUCCESSFUL"])

In [ ]:
# Split the preprocessed data into a training and testing dataset
X_traind3, X_testd3, y_traind3, y_testd3 = train_test_split(Xd3, yd3, random_state=42, stratify=y)


In [ ]:
# Create a StandardScaler instances
scalerd3 = StandardScaler()


In [ ]:
# Fit the StandardScaler
X_scalerd3 = scalerd3.fit(X_traind3)


In [ ]:
# Scale the data
X_train_scaledd3 = X_scalerd3.transform(X_traind3)
X_test_scaledd3 = X_scalerd3.transform(X_testd3)


In [ ]:
## Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaledd3,y_traind3,epochs=20,validation_data=(X_test_scaledd3,y_testd3))



In [ ]:
## Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values



In [ ]:
## Evaluate best model against full test data
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaledd3,y_testd3,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")